[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/NLTGit/OpenNightLights-colab-mirror/blob/master/onl/tutorials/mod3_5_expressions-PartA.ipynb)

# Expressions (10 min)

In the previous sub-module, {doc}`mod3_4_cell_statistics_band_math`, we executed some simple mathmatical operations on Images; however, sometimes we have multi-step operations or more complicated expressions we wish to apply.

For that, we can use the `.expression()` method built-in to the Image class.

In this tutorial we'll shift back to DMSP-OLS data and take a sneak peak at a process for intercalibrating DMSP data. As noted in {doc}`mod1_2_introduction_to_nighttime_light_data`, this is necessary when analyzing a DMSP-OLS time series. We get into this in much more detail in a later tutorial on DMSP-OLS intercalibration, but for now, we'll introduce the use of expressions in Google Earth Engine.

In order to manage memory for our interactive maps when publishing, we'll split this into two notebooks, A and B -- but consider them a single exercise.

**Our tasks in this exercise:**

Part A:
1. Invert a 1996 DMSP-OLS composite using built-in functions.

Part B:

2. Conduct an equivalent operation using `.expression()`
3. Apply a simple polynomial formula for DMSP-OLS intercalibration co-efficients to a 1996 composite.

## Invert an image with Image functions


### Initialize map with DMSP-OLS layer
First, as we've often done, let's initialize a map object.

Here we'll center our scene on Mexico City.

And we'll pull the annual DMSP-OLS nightime lights composite for 1996, using the "stable lights" band.

In [1]:
# !pip install geemap

In [2]:
# import geemap and ee for our Python session
import geemap, ee

try:
        ee.Initialize()
except Exception as e:
        ee.Authenticate()
        ee.Initialize()

# # initialize our map and center on Mexico City, Mexico
lat = 19.43
lon = -99.13
map1 = geemap.Map(center=[lat,lon],zoom=6)
map1.add_basemap('SATELLITE')

# get 1996 composite, apply mask, and add as layer
dmsp1996 = ee.Image("NOAA/DMSP-OLS/NIGHTTIME_LIGHTS/F121996").select('stable_lights')
map1.addLayer(dmsp1996.mask(dmsp1996), {}, "DMSP-OLS 1996", opacity=0.75)
map1.addLayerControl()
map1

Map(center=[19.43, -99.13], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButton(v…

### Invert an image

Let's say we want to invert an image. We know our max value (our Digital Number, if you recall) for DMSP-OLS images is 63. So if we multiply everything by -1 and then add 63. So, for example what was a DN of 63 before (the max) is now a 0 and vice versa.

We can do this using the built-in functions `.multiply` and `.add()`

**Note**: we selected our band, `stable_lights` above and saved to our variable `dmsp1996`, but if we didnt before we would have to do that here to explicitly apply our operation, including expression, to a specific band.

In [3]:
dmsp1996_inv = dmsp1996.multiply(-1).add(63)

### Visualize the inverse image

We'll view the transformed (inverted) image we just created and compare it to the original image with a split map.

**Note:** to ensure we're comparing them on the same scale, we're using the visual parameters in the `.addLayer()` function, which allows us to do a few things to our image, such as clip the image to "min" and "max" values.

In [4]:
map2 = geemap.Map(center=[lat,lon],zoom=6)
map2.addLayerControl()

dmsp96_tile = geemap.ee_tile_layer(dmsp1996, {'min':0,'max':63}, 'DMSP NTL 1996', opacity=0.75)
dmsp96inv_tile = geemap.ee_tile_layer(dmsp1996_inv, {'min':0,'max':63}, 'DMSP NTL 1996 inverse', opacity=0.75)

map2.split_map(left_layer=dmsp96_tile, right_layer=dmsp96inv_tile)

map2

Map(center=[19.43, -99.13], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButton(v…

Next we'll do this with the `expression()` method.

## References:
```{bibliography} ../references.bib
:filter: docname in docnames
```